# Objectives

# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score, precision_score
import xgboost as xgb
import joblib

# Dataset

In [2]:
df = pd.read_csv('..\models\df_balanced_full.csv', low_memory=False)
display(df)

,subject_id,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,10014078,103,103.0,°F,Temperature Fahrenheit,Routine Vital Signs,4.850000,2724,EW EMER.,Medicaid,UNABLE TO OBTAIN,EMERGENCY ROOM,60,-3,-3.0,mEq/L,Unknown,12.316667
1,10011398,25,25.0,mEq/L,TCO2 (calc) Arterial,Labs,7.900000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,1.4,1.4,Unknown,STAT,4.150000
2,10011398,25,25.0,mA,Temporary Ventricular Stim Setting mA,Cardiovascular (Pacer Data),16.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,384,384.0,mm Hg,Unknown,5.616667
3,10011398,36.8,36.8,°C,Temperature Celsius,Routine Vital Signs,18.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,98,98.0,%,Unknown,6.666667
4,10023771,100,100.0,%,O2 saturation pulseoxymetry,Respiratory,22.750000,2724,ELECTIVE,Medicare,WHITE,PHYSICIAN REFERRAL,70,1.2,1.2,Unknown,ROUTINE,15.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49249,10023117,104,104.0,mg/dL,Glucose (serum),Labs,9.050000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,EMERGENCY ROOM,58,2,2.0,Unknown,STAT,2.666667
49250,10019003,Obeys Commands,6.0,Unknown,GCS - Motor Response,Neurological,1.600000,Z87891,EW EMER.,Other,WHITE,TRANSFER FROM HOSPITAL,72,1.1,1.1,mg/dL,STAT,6.000000
49251,10023117,No response,1.0,Unknown,GCS - Motor Response,Neurological,4.950000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,___,133.0,mEq/L,STAT,2.566667
49252,10037861,No movement,0.0,Unknown,Strength L Arm,Neurological,23.433333,Z87891,EW EMER.,Medicare,UNKNOWN,EMERGENCY ROOM,79,21,21.0,mg/dL,STAT,9.183333


# Treatment

In [3]:
df = df.copy()

In [4]:
class Treatment(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()

        # Combined
        df['combined'] = np.where(
            (df['valueuom_chartevent'] == 'Unknown'),
            df['label_chartevent'],
            df['label_chartevent'] + ' (' + df['valueuom_chartevent'] + ')'
        )

        # Feature Interactions
        urgency_rank = {
            'ELECTIVE': 1, 'OBSERVATION ADMIT': 2,
            'SURGICAL SAME DAY ADMISSION': 3, 'URGENT': 4,
            'DIRECT EMER.': 5, 'EW EMER.': 6
        }
        df['urgency_score'] = df['admission_type'].map(urgency_rank)
        df['urgency_x_lab_delay'] = df['urgency_score'] * df['time_since_admission_labevent']
        df['admission_x_age'] = df['urgency_score'] * df['age']

        #print("Fisrt display Combine + FI")
        #display(df.head())

        # OneHot Encoding
        variables_cat = ['admission_type', 'insurance', 'priority']
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        one_hot_encoded = encoder.fit_transform(df[variables_cat])

        one_hot_df = pd.DataFrame(one_hot_encoded, 
                          columns=encoder.get_feature_names_out(variables_cat),
                          index=df.index)

        df = pd.concat([df.drop(columns=variables_cat), one_hot_df], axis=1)
        #print("Second display OneHot enconding")
        #display(df.head())
        
        # Label Encoding
        variables = ['value_chartevent', 'label_chartevent', 'valueuom_labevent',
                     'valueuom_chartevent', 'admission_location', 'race', 'category']

        for col in variables:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col].astype(str))
            else:
                print(f"Aviso: Coluna '{col}' não encontrada.")



        #print("Thrid Display Label enconding:")
        #display(df.head())

        # Frequency
        df['value_labevent'] = df['value_labevent'].astype(str)
        df['value_labevent_freq'] = df['value_labevent'].map(df['value_labevent'].value_counts())
        df.drop(columns=['value_labevent'], inplace=True)

        df['combined'] = df['combined'].astype(str)
        df['combined_freq'] = df['combined'].map(df['combined'].value_counts())
        df.drop(columns=['combined'], inplace=True)

        #print("Fourth Display Label enconding:")
        #display(df.head())

        # Drop colunas
        #cols_to_drop = ['subject_id', 'valueuom_chartevent','label_chartevent','valueuom_labevent']
        #df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        #print(df.dtypes)

        #print("Final")
        #display(df)

        return df

# Model

In [5]:

le = LabelEncoder()
y = le.fit_transform(df['icd_code'])
X = df.drop('icd_code', axis=1)


In [6]:
treatment = Treatment()
X_processed = treatment.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, stratify=y, test_size=0.2, random_state=42)

In [8]:

param_dist = {
    'model__max_depth': [5],
    'model__learning_rate': [0.05],
    'model__gamma': [0.5],
    'model__min_child_weight': [3],
    'model__reg_alpha': [0],
    'model__reg_lambda': [1],
    'model__n_estimators': [200]  
}


pipeline = Pipeline([
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(le.classes_),
        tree_method='hist',
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42
    ))
])

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_dist,
    scoring='precision_weighted',
    cv=3,
    n_jobs=-1,
    verbose=2 )

In [9]:
# === Treinar === #
grid_search.fit(X_train, y_train)

# === Resultados === #
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


C:\Users\catar\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [17:45:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best Parameters: {'model__gamma': 0.5, 'model__learning_rate': 0.05, 'model__max_depth': 5, 'model__min_child_weight': 3, 'model__n_estimators': 200, 'model__reg_alpha': 0, 'model__reg_lambda': 1}
Accuracy: 0.7238241933396488


In [11]:

y_pred = grid_search.predict(X_test)

print("\nAccuracy Test", accuracy_score(y_test, y_pred))
print("Precision (macro):", precision_score(y_test, y_pred, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy Test 0.6612526647040909
Precision (macro): 0.7443562338073063

Classification Report:
               precision    recall  f1-score   support

        2724       0.65      0.95      0.77      1641
        4019       0.91      0.49      0.64      1642
        E039       0.68      0.50      0.58      1642
        E785       0.99      0.14      0.25      1642
        Z794       0.72      1.00      0.84      1642
      Z87891       0.51      0.88      0.65      1642

    accuracy                           0.66      9851
   macro avg       0.74      0.66      0.62      9851
weighted avg       0.74      0.66      0.62      9851



In [13]:

joblib.dump(grid_search.best_estimator_, 'modelo_pipeline_final.pkl')

['modelo_pipeline_final.pkl']

# Example

In [26]:
import joblib

In [27]:
pipeline = joblib.load('modelo_pipeline_final.pkl')

In [28]:
cases = pd.read_csv('..\models\cases.csv')

In [29]:
treatment = Treatment()
case = treatment.transform(cases)

In [30]:
preds = pipeline.predict(case)

ValueError: feature_names mismatch: ['subject_id', 'value_chartevent', 'valuenum_chartevent', 'valueuom_chartevent', 'label_chartevent', 'category', 'time_since_admission_chartevent', 'race', 'admission_location', 'age', 'valuenum_labevent', 'valueuom_labevent', 'time_since_admission_labevent', 'urgency_score', 'urgency_x_lab_delay', 'admission_x_age', 'admission_type_DIRECT EMER.', 'admission_type_ELECTIVE', 'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT', 'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Other', 'priority_ROUTINE', 'priority_STAT', 'priority_Unknown', 'value_labevent_freq', 'combined_freq'] ['subject_id', 'value_chartevent', 'valuenum_chartevent', 'valueuom_chartevent', 'label_chartevent', 'category', 'time_since_admission_chartevent', 'race', 'admission_location', 'age', 'valuenum_labevent', 'valueuom_labevent', 'time_since_admission_labevent', 'urgency_score', 'urgency_x_lab_d

In [ ]:
print("Predições:", preds)